In [2]:
import pandas as pd

In [3]:
# there are ratios in the data, can they be used?
# (I dont think interpretations alone are going to cut it)
# So what can I do with these ratios?

In [4]:
# ooh, if I can find something like LCR(ofc, excepting other ratios such as quick), then I can generate a 
# basic basel report> bool:compliant/not
# you dont really need uslocs for this cos you dont need large time data, you are calculating assets for a point
# so, use balance sheets

In [5]:
"""I can do simple metrics calculaitons and maybe predict them, but the regression does not seem to be working"""

'I can do simple metrics calculaitons and maybe predict them, but the regression does not seem to be working'

In [6]:
# quick ratio, cash ratio, net working capital ratio, curretn ratio vs liq ratio

#Tier 1 (core) risk-based capital
#Tier 2 risk-based capital
#Equity capital to assets
#Core capital (leverage) ratio

In [7]:
# commonstk = 3020000000
# toteqtycap = 236206000000
# riskwtd_assets = 1394763000000
# toteqtycap/riskwtd_assets * 100

## Basic Stats Analysis

In [8]:
df = pd.read_csv("../../data/raw/fin_statements/bofa_bs.csv")

In [9]:
df.head(10)
"""Last col -20 is all wrong, actually -grey cols"""

'Last col -20 is all wrong, actually -grey cols'

### Liquidity

In [10]:
ca = ['Cash, Cash Equivalents & Federal Funds Sold',  'Available for Sale Securities',
      'Trading Securities', 'Other Short Term Investments', 'Loans Held for Sale', 'Accounts receivable', 
      'Other Receivables', 'Other Assets', 'Derivative Assets', 'Held To Maturity Securities']

# cce: 'Cash And Cash Equivalents', 'Cash', 'Cash And Due from Banks','Interest Bearing Deposits Assets', 'Money Market Investments',

 # this includes cash, cash eq, marketable securities, accs recievable
# I am not really seeing prepaid expenses or inventory on the balance sheet - they all seem quick enough to me
# or shld I split qa along trading securities, other short term inv, loans held for sale
qa = ['Cash, Cash Equivalents & Federal Funds Sold', 'Available for Sale Securities',
      'Trading Securities', 'Accounts receivable']



cl = ['Total Deposits','Federal Funds Purchased And Securities Sold Under Agreement To Repurchase', 'Current Accrued Expenses',
      'Current Debt And Capital Lease Obligation', 'Trading Liabilities', 'Derivative Product Liabilities', 'Other Liabilities']
# ambiguous = derviative assets, HMS, 


#### Quick ratio & Current Ratio

In [11]:
def get_qancl(df): # all rows of quick a and curr liab, ca for all banks
    quick_assets = []
    current_liabilities = []
    current_assets = []
    for idx, row in df.iterrows():
        if row['Breakdown'] in qa:
            quick_assets.append(row)
        if row["Breakdown"] in cl:
            current_liabilities.append(row)
        if row["Breakdown"] in ca:
            current_assets.append(row)
    return quick_assets, current_assets, current_liabilities
    
    # len(current_assets), len(current_liabilities)


In [12]:
def get_qancl_yr(yr, bank, quick_assets, current_assets, current_liabilities): # total canl for yr 
    # pass yr as index : '23-2, '22 - 3, '21-4, '20-5 
    qa_yr = 0
    cl_yr = 0
    ca_yr = 0
    for asset in quick_assets:
        if asset[1] == bank:
            print(asset[yr])
            val = int(asset[yr].replace(',', "").replace("-", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(asset[i].replace(',', "")) 
                val = val/3
            qa_yr += val

    
    for lia in current_liabilities:
        if asset[1] == bank:
            val = int(lia[2].replace(',', "").replace("-", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(asset[i].replace(',', ""))
                val = val/3
            cl_yr += val

    for asset in current_assets:
        if asset[1] == bank:
            val = int(asset[yr].replace(',', "").replace("-", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(asset[i].replace(',', "")) 
                val = val/3
            ca_yr += val

    return qa_yr*1000, ca_yr*1000, cl_yr*1000  # for yr for bank
# all yf values are in thousands



In [13]:
quick_assets, current_assets, current_liabilities = get_qancl(df)
qa_23, ca_23, cl_23 = get_qancl_yr(4, 'BAC', quick_assets, current_assets, current_liabilities)
quick_ratio = qa_23/cl_23 
current_ratio = ca_23/cl_23

606,085,000
143,646,000
8,895,000
72,263,000


In [14]:
quick_ratio, qa_23/10**9, cl_23/10**9

(0.32866746899606525, 830.889, 2528.0536663333337)

In [15]:
current_ratio, ca_23/10**9

(0.8455754039036849, 2137.66)

In [16]:
# the grey cols are wrong - rescrape and check incom, cash flow also :===

#### Other Ratios

In [ ]:
# get from data directly and analyse

## Compliance

### CAR - Capital Adequacy
*Under Basel III, all banks are required to have a Capital Adequacy Ratio of at least 8%*

In [3]:
basic = pd.read_csv("../../data/interim/US_Banks_Basic.csv")

In [4]:
t1 = '"Tier 1 (core) risk-based capital"'
t2 = '"Tier 2 risk-based capital"'
rwa = '"Total risk weighted assets"'

In [5]:
# for all banks
def get_car():
    queries = [t1, t2, rwa]
    banks = basic.bank.unique()
    car = pd.DataFrame()
    for query in queries:
        car_temp = basic.query(f"tag == {query}")
        car = pd.concat([car, car_temp], ignore_index=True)
    # for bank in banks:# calc car
    bank_cars = {}
    for idx, row in car.iterrows():
        b = row['bank']
        if b not in bank_cars:
            bank_cars[b] = row.iloc[2:]
        else:
            if row['tag'] == "Tier 2 risk-based capital":
                tier2 = row.iloc[2:]
                num = bank_cars[b]
                bank_cars[b] = [num[i] + tier2[i] for i in range(len(tier2))]
            if row['tag'] == 'Total risk weighted assets':
                weighted = row.iloc[2:]
                num = bank_cars[b]
                bank_cars[b] = [num[i] / weighted[i] for i in range(len(tier2))]
                
    return car, bank_cars
car_df, bank_cars = get_car()


In [6]:
(bank_cars['jpmorgan-chase-bank'][:6])

[0.17345533056157378,
 0.18453470338218325,
 0.18175139303629784,
 0.1844286266329814,
 0.1806011052693971,
 0.17477219653253404]

In [11]:
def car_comp(basic, bank, qtr):
    car_df, bank_cars = get_car()
    cars = bank_cars[bank]

    qtrs = basic.columns[2:].to_list()
    qtr_idx = qtrs.index(f'{qtr}')

    if cars[qtr_idx]*100 > 8:
        print(f"Compliant: {cars[qtr_idx]*100}%")
    else:
        print(f"Not Compliant: {cars[qtr_idx]*100}%")
        

bank = 'jpmorgan-chase-bank'
qtr = "2002-12-31"
car_comp(basic, bank, qtr)

Compliant: 11.127689518782852%


### But, Is the Bank Large Enough?
*Under the Federal Reserve Board's capital framework for bank holding companies, covered savings and loan holding companies, and U.S. intermediate holding companies with $100 billion or more in total consolidated assets, capital requirements are in part determined by the supervisory stress test results.*

In [95]:
def check_large(bank):
    totalAssets = basic.query(f"tag == 'Total Assets:'")
    isLarge = {}
    for idx, row in totalAssets.iterrows():
        b = row['bank']
        assets = row.iloc[2:]
        bools = []
        for asset in assets:
            if asset > 100*10**9:
                bools.append('L')
            else:
                bools.append('NL')
        isLarge[b] = bools
    return isLarge[bank]
isLarge = check_large('jpmorgan-chase-bank')


### The Capital Ratios 

*Under the guidelines, banking organizations are required to maintain a minimum Total risk-based capital ratio (total capital to risk-weighted assets) of 8% and a Tier 1 risk-based capital ratio of 4%.*

In [124]:
t1cr = '"Tier 1 risk-based capital ratio"'
t1lever = '"Core capital (leverage) ratio"'
totCapR = '"Total risk-based capital ratio"'
def get_capr():
    queries = [t1cr, t1lever, totCapR]
    ratios = pd.DataFrame()
    for query in queries:
        r_temp = basic.query(f"tag == {query}")
        ratios = pd.concat([ratios,r_temp], ignore_index=True)
    return ratios

rats = get_capr()

def isCompliant(ratio_df, bank):
    isLarge = check_large(bank)
    bank_r = ratio_df.query(f"bank == '{bank}'")
    t1crcomp = []
    t1lcomp = []
    totcomp = []
    for idx, row in bank_r.iterrows():
        if row["tag"] == 'Tier 1 risk-based capital ratio':
            t1r = row.iloc[2:]
            for i in range(len(t1r)):
                if isLarge[i] == 'L':
                    if t1r[i] > 6: # basel 3 norms
                        t1crcomp.append(True)
                    else:
                        t1crcomp.append(False)
                else:
                    t1crcomp.append('notLarge')
    
        if row["tag"] == "Core capital (leverage) ratio": # > 5%
            t1r = row.iloc[2:]
            
            for i in range(len(t1r)):
                if isLarge[i] == 'L':
                    if t1r[i] > 5:
                        t1lcomp.append(True)
                    else:
                        t1lcomp.append(False)
                else:
                    t1lcomp.append('notLarge')
                
        if row["tag"] == "Total risk-based capital ratio": 
            t1r = row.iloc[2:]
            for i in range(len(t1r)):
                if isLarge[i] == 'L':
                    if t1r[i] > 8:
                        totcomp.append(True)
                    else:
                        totcomp.append(False)
                else:
                    totcomp.append('notLarge')
        
    return t1crcomp, totcomp, t1lcomp


In [131]:
def get_comp(ratio_df, bank, qtr):
    t1, tot, t1l = isCompliant(ratio_df, f'{bank}')
    qtrs = ratio_df.columns[2:].to_list()
    qtr_idx = qtrs.index(f'{qtr}')
    print( f"Tier 1 Capital Ratio(RB): {t1[qtr_idx]}, Total Capital Ratio(RB): {tot[qtr_idx]}, Tier 1 Leverage Ratio: {t1l[qtr_idx]}")


ratio_df = get_capr()
qtr = "2023-06-30"
bank = "sumitomo-mitsui-trust-bank-u-s-a-limited"
get_comp(ratio_df, bank, qtr)

Tier 1 Capital Ratio(RB): notLarge, Total Capital Ratio(RB): notLarge, Tier 1 Leverage Ratio: notLarge


In [127]:
banks = basic.bank.unique()
banks

array(['jpmorgan-chase-bank', 'bank-of-america', 'u-s-bank',
       'goldman-sachs-bank-usa', 'hsbc-bank-usa', 'morgan-stanley-bank',
       'mitsubishi-ufj-trust-banking-corporation-u-s-a', 'pnc-bank',
       'charles-schwab-bank-ssb',
       'sumitomo-mitsui-trust-bank-u-s-a-limited', 'truist-bank',
       'wells-fargo-bank', 'td-bank', 'the-huntington-national-bank',
       'citizens-bank-57957', 'citibank', 'fifth-third-bank',
       'regions-bank', 'comerica-bank', 'american-express-national-bank',
       'capital-one', 'the-bank-of-new-york-mellon'], dtype=object)

## Extract Ratios

In [33]:
basic = pd.read_csv("../../data/raw/uslocs/banks_basic_final.csv")
basic.head(2)

,Tag,Bank,2023-12-31,2023-09-30,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,...,2002-12-31,2001-12-31,2000-12-31,1999-12-31,1998-12-31,1997-12-31,1996-12-31,1995-12-31,1994-12-31,1993-12-31
0,# of Employees:,jpmorgan-chase-bank,"226,822","226,649","219,830","221,852","220,271","216,524","210,348","207,949",...,"77,286","67,401","56,567","41,563","42,229","41,670","41,615","24,042","25,546","26,409"
1,Total Assets:,jpmorgan-chase-bank,"$3,395,126,000,000","$3,385,581,000,000","$3,382,195,000,000","$3,267,963,000,000","$3,201,942,000,000","$3,308,575,000,000","$3,380,824,000,000","$3,476,711,000,000",...,"$621,962,000,000","$537,826,000,000","$377,116,000,000","$332,198,000,000","$296,717,000,000","$297,061,000,000","$272,429,000,000","$147,120,000,000","$135,742,000,000","$115,510,000,000"


In [34]:
def date_to_qtr(date):
    yr = date[:4]
    q = date[5:7]
    if q == '12':
        qtr = 'Q4'
    if q == '09':
        qtr = 'Q3'
    if q == '06':
        qtr = 'Q2'
    if q == '03':
        qtr = 'Q1'    
    return (yr + " " + qtr)

In [35]:
cols = basic.columns[2:]
d2q = {col : date_to_qtr(col) for col in cols}
basic = basic.rename(columns=d2q)

In [36]:
basic = basic.fillna('0')

In [37]:
basic.head(2)

,Tag,Bank,2023 Q4,2023 Q3,2023 Q2,2023 Q1,2022 Q4,2022 Q3,2022 Q2,2022 Q1,...,2002 Q4,2001 Q4,2000 Q4,1999 Q4,1998 Q4,1997 Q4,1996 Q4,1995 Q4,1994 Q4,1993 Q4
0,# of Employees:,jpmorgan-chase-bank,"226,822","226,649","219,830","221,852","220,271","216,524","210,348","207,949",...,"77,286","67,401","56,567","41,563","42,229","41,670","41,615","24,042","25,546","26,409"
1,Total Assets:,jpmorgan-chase-bank,"$3,395,126,000,000","$3,385,581,000,000","$3,382,195,000,000","$3,267,963,000,000","$3,201,942,000,000","$3,308,575,000,000","$3,380,824,000,000","$3,476,711,000,000",...,"$621,962,000,000","$537,826,000,000","$377,116,000,000","$332,198,000,000","$296,717,000,000","$297,061,000,000","$272,429,000,000","$147,120,000,000","$135,742,000,000","$115,510,000,000"


In [38]:
percentages = pd.DataFrame(columns=basic.columns) # for 1 bank, also appending rows cos its small
bank = 'jpmorgan-chase-bank'
for idx, row in basic.iterrows():
    
    if row['Bank'] == bank:
        val = row['2023 Q4']
        if val.find('%') != -1:
            percentages.loc[percentages.shape[0]] = row.to_list()
            # percentages = pd.concat([percentages, n_df], ignore_index=True, axis=0)
percentages.head()

,Tag,Bank,2023 Q4,2023 Q3,2023 Q2,2023 Q1,2022 Q4,2022 Q3,2022 Q2,2022 Q1,...,2002 Q4,2001 Q4,2000 Q4,1999 Q4,1998 Q4,1997 Q4,1996 Q4,1995 Q4,1994 Q4,1993 Q4
0,% insured,jpmorgan-chase-bank,39.75%,40.86%,41.00%,39.20%,39.39%,38.91%,38.10%,37.08%,...,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
1,Percentage of unprofitable institutions,jpmorgan-chase-bank,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Yield on earning assets,jpmorgan-chase-bank,5.1189%,4.9590%,4.7678%,4.5497%,2.8446%,2.4275%,2.0780%,1.8937%,...,0,0,0,0,0,0,0,0,0,0
3,Cost of funding earning assets,jpmorgan-chase-bank,1.9710%,1.8481%,1.6997%,1.5120%,0.5278%,0.3192%,0.1539%,0.0796%,...,0,0,0,0,0,0,0,0,0,0
4,Net interest margin,jpmorgan-chase-bank,3.1479%,3.1109%,3.0680%,3.0377%,2.3167%,2.1083%,1.9241%,1.8141%,...,0,0,0,0,0,0,0,0,0,0


In [39]:
percentages['Tag']

0                                             % insured
1               Percentage of unprofitable institutions
2                               Yield on earning assets
3                        Cost of funding earning assets
4                                   Net interest margin
5                  Noninterest income to average assets
6                 Noninterest expense to average assets
7               Loan and lease loss provision to assets
8                        Net operating income to assets
9                                Return on assets (ROA)
10                              Pretax return on assets
11                               Return on Equity (ROE)
12       Retained earnings to average equity (ytd only)
13                             Net charge-offs to loans
14             Credit loss provision to net charge-offs
15             Earnings coverage of net charge-offs (x)
16                                     Efficiency ratio
17                      Assets per employee ($mi